# A parser for NOAA bulletins

In [20]:
import feedparser

In [21]:
import numpy as np
import pandas as pd

In [22]:
import urllib

In [23]:
import re

In [24]:
from Storms.utils import readxml

In [25]:
url='http://www.nhc.noaa.gov/index-at.xml'

In [26]:
url='http://www.nhc.noaa.gov/index-ep.xml'

In [27]:
atl=feedparser.parse(url)

In [28]:
data=atl['entries']
try:
    data[0]['summary']
except:
    print data

In [29]:
print readxml(url,'lxml')

Mon, 08 May 2017 10:31:57 GMT
NHC Eastern North Pacific
Active tropical cyclones in the Eastern North Pacific
http://www.nhc.noaa.gov/
none
nhcwebmaster@noaa.gov (NHC Webmaster)
en-us
nhcwebmaster@noaa.gov (NHC Webmaster)
http://www.nhc.noaa.gov/gifs/xml_logo_nhc.gif
http://www.nhc.noaa.gov/
NHC Eastern North Pacific
NOAA logo
95
45
Eastern North Pacific Tropical Weather Outlook
000
ABPZ20 KNHC 071846
TWOEP
Special Tropical Weather Outlook
NWS National Hurricane Center Miami FL
1145 AM PDT Sun May 7 2017
For the eastern North Pacific...east of 140 degrees west longitude:
An area of of disturbed weather has developed in association with a
low pressure area a few hundred miles southwest of the coast of
Costa Rica.
Environmental conditions are forecast to be favorable
for gradual development, and a tropical depression could form by the
middle of next week while the low moves slowly toward the northwest.
The next Special Tropical Weather Outlook will be issued by 12 PM
PDT Monday.
* Format

In [30]:
data

[{'author': u'nhcwebmaster@noaa.gov (NHC Webmaster)',
  'author_detail': {'email': u'nhcwebmaster@noaa.gov',
   'name': u'NHC Webmaster'},
  'authors': [{'email': u'nhcwebmaster@noaa.gov', 'name': u'NHC Webmaster'}],
  'guidislink': False,
  'id': u'http://www.nhc.noaa.gov/gtwo.php?basin=epac&201705071846',
  'link': u'http://www.nhc.noaa.gov/gtwo.php?basin=epac',
  'links': [{'href': u'http://www.nhc.noaa.gov/gtwo.php?basin=epac',
    'rel': u'alternate',
    'type': u'text/html'}],
  'published': u'Sun, 07 May 2017 18:46:19 GMT',
  'published_parsed': time.struct_time(tm_year=2017, tm_mon=5, tm_mday=7, tm_hour=18, tm_min=46, tm_sec=19, tm_wday=6, tm_yday=127, tm_isdst=0),
  'summary': u'<br />\n000<br />\nABPZ20 KNHC 071846<br />\nTWOEP <br />\n<br />\nSpecial Tropical Weather Outlook<br />\nNWS National Hurricane Center Miami FL<br />\n1145 AM PDT Sun May 7 2017<br />\n<br />\nFor the eastern North Pacific...east of 140 degrees west longitude:<br />\n<br />\nAn area of of disturbed 

In [31]:
buls = len(data)

In [32]:
for b in range(buls):
    print (data[b].summary)

<br />
000<br />
ABPZ20 KNHC 071846<br />
TWOEP <br />
<br />
Special Tropical Weather Outlook<br />
NWS National Hurricane Center Miami FL<br />
1145 AM PDT Sun May 7 2017<br />
<br />
For the eastern North Pacific...east of 140 degrees west longitude:<br />
<br />
An area of of disturbed weather has developed in association with a<br />
low pressure area a few hundred miles southwest of the coast of<br />
Costa Rica.  Environmental conditions are forecast to be favorable<br />
for gradual development, and a tropical depression could form by the<br />
middle of next week while the low moves slowly toward the northwest.<br />
The next Special Tropical Weather Outlook will be issued by 12 PM<br />
PDT Monday.<br />
* Formation chance through 48 hours...low...near 20 percent.<br />
* Formation chance through 5 days...high...70 percent.<br />
<br />
A second area of disturbed weather has formed several hundred<br />
miles south-southwest of Acapulco, Mexico.  Some slow development<br />
c

## retrieve from source

In [33]:
url = 'https://metoc.ndbc.noaa.gov/RSSFeeds-portlet/img/jtwc/jtwc.rss'

In [34]:
print readxml(url,'lxml')

JTWC TROPICAL CYCLONE INFORMATION FEED
https://metoc.ndbc.noaa.gov/web/guest/jtwc
JTWC Tropical Cyclone Information
Mon, 08 May 2017 11:12:08 +0000
Mon, 08 May 2017 11:12:08 +0000
ContentFeeder 2.0
https://metoc.ndbc.noaa.gov/web/guest/jtwc
JTWC TROPICAL CYCLONE INFORMATION FEED
https://metoc.ndbc.noaa.gov/web/guest/jtwc
Current Northwest Pacific/North Indian Ocean* Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
No Current Tropical Cyclone Warnings.
* Includes Bay of Bengal and Arabian Sea
]]>
CDO.JTWC.fct@navy.mil (JTWC CDO)
Northwest Pacific/North Indian Ocean* Tropical Systems
NWPAC-NIO-WARNINGS
Mon, 08 May 17 11:12:08 +0000
Current Central/Eastern Pacific Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
No Current Tropical Cyclone Warnings.
]]>
CDO.JTWC.fct@navy.mil (JTWC CDO)
Central/Eastern Pacific Tropical Systems
EPAC-CPAC-WARNINGS
Mon, 08 May 17 11:12:08 +0000
Current Southern Hemisphere Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
Tropic

In [35]:
jtwc=feedparser.parse(url) # parse the webpage

In [36]:
# collect the summeries (one for each alert present in the webpage)
data=jtwc['entries']
for k in range(len(data)):
    if 'tcw' in data[k].summary : txt = (data[k].summary)

In [37]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('Tropical')] # we split the text to get the references for the alerts

In [38]:
hur = [elem.strip().split('\n')[0] for elem in elp] # the names of the TCs are...

In [39]:
hur

[u'', u'Cyclone  18P (Donna) Warning #19 ']

In [40]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('href=')] # we now split differently to get the urls for the bulletins

In [41]:
bul = [elem for elem in elp if 'tcw' in elem] # collect the urls 

bul_ =  [elem.strip().split()[0] for elem in bul]

bul_

[u'"https://metoc.ndbc.noaa.gov/ProductFeeds-portlet/img/jtwc/products/sh1817.tcw"']

In [42]:
bul_ = [ elem.encode('utf8').replace('"', '') for elem in bul_] # get rid of unicode characters

In [44]:
det = urllib.urlopen(bul_[0]).read() # download and parse the bulletin file

In [45]:
print det

WTPS51 PGTW 080900    
WARNING    ATCG MIL 18P SWP 170508072341
2017050806 18P DONNA      019  01 155 07 SATL 020
T000 175S 1651E 115 R064 030 NE QD 035 SE QD 035 SW QD 030 NW QD R050 055 NE QD 045 SE QD 055 SW QD 040 NW QD R034 140 NE QD 180 SE QD 160 SW QD 110 NW QD 
T012 188S 1658E 110 R064 045 NE QD 045 SE QD 045 SW QD 030 NW QD R050 085 NE QD 095 SE QD 075 SW QD 055 NW QD R034 165 NE QD 195 SE QD 160 SW QD 110 NW QD 
T024 203S 1666E 090 R064 030 NE QD 030 SE QD 030 SW QD 015 NW QD R050 065 NE QD 075 SE QD 060 SW QD 040 NW QD R034 160 NE QD 180 SE QD 125 SW QD 095 NW QD 
T036 218S 1677E 065 R064 020 NE QD 025 SE QD 025 SW QD 020 NW QD R050 045 NE QD 045 SE QD 035 SW QD 025 NW QD R034 160 NE QD 170 SE QD 105 SW QD 080 NW QD 
T048 237S 1693E 050 R050 020 NE QD 010 SE QD 015 SW QD 010 NW QD R034 145 NE QD 160 SE QD 085 SW QD 055 NW QD 
T072 282S 1740E 035 R034 165 NE QD 175 SE QD 050 SW QD 055 NW QD 
AMP
    036HR BECOMING EXTRATROPICAL
    048HR BECOMING EXTRATROPICAL
    072HR EXTRA

In [46]:
#select the first lines where all the info is (conviniently) stored
data =[line.strip() for line in det.splitlines() if ('T' is line.strip()[0]) & ('QD' in line.strip()[-2:])]

In [47]:
data = [re.sub(' ',',',elem) for elem in data] # replace space with commas as delimiter

In [48]:
data = [v.split(',') for v in data] #split to array 

In [49]:
df = pd.DataFrame(data) # create a dataframe

In [50]:
df.head() #check

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,T000,175S,1651E,115,R064,030,NE,QD,035,SE,...,QD,180,SE,QD,160,SW,QD,110,NW,QD
1,T012,188S,1658E,110,R064,045,NE,QD,045,SE,...,QD,195,SE,QD,160,SW,QD,110,NW,QD
2,T024,203S,1666E,090,R064,030,NE,QD,030,SE,...,QD,180,SE,QD,125,SW,QD,095,NW,QD
3,T036,218S,1677E,065,R064,020,NE,QD,025,SE,...,QD,170,SE,QD,105,SW,QD,080,NW,QD
4,T048,237S,1693E,050,R050,020,NE,QD,010,SE,...,None,None,None,None,None,None,None,None,None,None


In [52]:
# drop the text columns [NE,QD,....]
#df = df.drop(df.columns[[6,7,9,10,12,13,15,16,19,20,22,23,25,26,28,29,32,33,35,36,38,39,41,42]], axis=1) 
for ref in ['NE','NW','SW','SE','QD']:
    df = df.loc[:, (df != [ref]).all(axis=0)] 

In [53]:
df = df.set_index(df.columns[0]) # set time as index

In [54]:
df.head() #check

,1,2,3,4,5,8,11,14,17,18,21,24,27,30,31,34,37,40
0,,,,,,,,,,,,,,,,,,
T000,175S,1651E,115,R064,030,035,035,030,R050,055,045,055,040,R034,140,180,160,110
T012,188S,1658E,110,R064,045,045,045,030,R050,085,095,075,055,R034,165,195,160,110
T024,203S,1666E,090,R064,030,030,030,015,R050,065,075,060,040,R034,160,180,125,095
T036,218S,1677E,065,R064,020,025,025,020,R050,045,045,035,025,R034,160,170,105,080
T048,237S,1693E,050,R050,020,010,015,010,R034,145,160,085,055,None,None,None,None,None


In [55]:
df.columns=np.arange(df.shape[1]) # rename the columns

In [56]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,,
T000,175S,1651E,115,R064,030,035,035,030,R050,055,045,055,040,R034,140,180,160,110
T012,188S,1658E,110,R064,045,045,045,030,R050,085,095,075,055,R034,165,195,160,110
T024,203S,1666E,090,R064,030,030,030,015,R050,065,075,060,040,R034,160,180,125,095
T036,218S,1677E,065,R064,020,025,025,020,R050,045,045,035,025,R034,160,170,105,080
T048,237S,1693E,050,R050,020,010,015,010,R034,145,160,085,055,None,None,None,None,None


In [57]:
#move the values to appropiate place based on the wind radii in order to force [64,50,34].

for i in range(df.shape[0]):
    if df.iloc[i,3]=='R034' : 
        df.iloc[i,13:] = df.iloc[i,3:8].values
        df.iloc[i,3:8] = None
    elif df.iloc[i,3]=='R050' :
        df.iloc[i,8:] = df.iloc[i,3:13].values
        df.iloc[i,3:8] = None


In [58]:
df #check

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,,
T000,175S,1651E,115,R064,030,035,035,030,R050,055,045,055,040,R034,140,180,160,110
T012,188S,1658E,110,R064,045,045,045,030,R050,085,095,075,055,R034,165,195,160,110
T024,203S,1666E,090,R064,030,030,030,015,R050,065,075,060,040,R034,160,180,125,095
T036,218S,1677E,065,R064,020,025,025,020,R050,045,045,035,025,R034,160,170,105,080
T048,237S,1693E,050,None,None,None,None,None,R050,020,010,015,010,R034,145,160,085,055
T072,282S,1740E,035,None,None,None,None,None,None,None,None,None,None,R034,165,175,050,055


In [59]:
df = df.drop(df.columns[[3,8,13]], axis=1) # drop text columns [R064,R050,R034]

In [60]:
#set columns
df.columns=['lat','lon','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [61]:
df #check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,,,,,,,,,,,,,,,
T000,175S,1651E,115,030,035,035,030,055,045,055,040,140,180,160,110
T012,188S,1658E,110,045,045,045,030,085,095,075,055,165,195,160,110
T024,203S,1666E,090,030,030,030,015,065,075,060,040,160,180,125,095
T036,218S,1677E,065,020,025,025,020,045,045,035,025,160,170,105,080
T048,237S,1693E,050,None,None,None,None,020,010,015,010,145,160,085,055
T072,282S,1740E,035,None,None,None,None,None,None,None,None,165,175,050,055


In [62]:
tidx = [np.float(elem[1:]) for elem in df.index.values] #convert time from string to float 

In [63]:
df.index=tidx # set float time as index

In [64]:
df # check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0.0,175S,1651E,115,030,035,035,030,055,045,055,040,140,180,160,110
12.0,188S,1658E,110,045,045,045,030,085,095,075,055,165,195,160,110
24.0,203S,1666E,090,030,030,030,015,065,075,060,040,160,180,125,095
36.0,218S,1677E,065,020,025,025,020,045,045,035,025,160,170,105,080
48.0,237S,1693E,050,None,None,None,None,020,010,015,010,145,160,085,055
72.0,282S,1740E,035,None,None,None,None,None,None,None,None,165,175,050,055
